In [1]:
import pandas as pd
import numpy as np

Data preprocessing

In [2]:
aluminium = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/ALUMINIUM.csv')
brent = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/BRENT.csv')
cocoa = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/COCOA.csv')
cofee = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/COFEE.csv')
copper = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/COPPER.csv')
corn = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/CORN.csv')
cotton = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/COTTON.csv')
gasoline = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/GASOLINE.csv')
gold = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/GOLD.csv')
nickel = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/NICKEL.csv')
palladium = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/PALLADIUM.csv')
platinum = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/PLATINUM.csv')
silver = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/SILVER.csv')
soybean = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/SOYBEAN.csv')
wheat = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/WHEAT.csv')
wti = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/WTI.csv')
zinc = pd.read_csv('/Users/nickarhi/Desktop/Studying/Project HSE 4th course/Data/ZINC.csv')

In [3]:

cocoa = cocoa.drop(['Open', 'High', 'Low','Dividends','Stock Splits'], axis=1)
copper = copper.drop(['Open', 'High', 'Low','Dividends','Stock Splits'], axis=1)
corn = corn.drop(['Open', 'High', 'Low','Dividends','Stock Splits'], axis=1)
cotton = cotton.drop(['Open', 'High', 'Low','Dividends','Stock Splits'], axis=1)
gold = gold.drop(['Open', 'High', 'Low','Dividends','Stock Splits'], axis=1)
cofee = cofee.drop(['Open', 'High', 'Low', 'Dividends','Stock Splits'], axis=1)
palladium = palladium.drop(['Open', 'High', 'Low', 'Dividends','Stock Splits'], axis=1)
platinum = platinum.drop(['Open', 'High', 'Low', 'Dividends','Stock Splits'], axis=1)
silver = silver.drop(['Open', 'High', 'Low', 'Dividends','Stock Splits'], axis=1)
soybean = soybean.drop(['Open', 'High', 'Low', 'Dividends','Stock Splits'], axis=1)
wheat = wheat.drop(['Open', 'High', 'Low', 'Dividends','Stock Splits'], axis=1)

aluminium = aluminium.drop(['Open', 'High', 'Low'], axis=1)
zinc = zinc.drop(['Open', 'High', 'Low'], axis=1)
nickel = nickel.drop(['Open', 'High', 'Low'], axis=1)
gasoline = gasoline.drop(['Open', 'High', 'Low'], axis=1)
brent = brent.drop(['Open', 'High', 'Low'], axis=1)
wti = wti.drop(['Open', 'High', 'Low'], axis=1)


In [4]:
def convert_date(date):
    return pd.to_datetime(date).date()

def preprocess_df(df):
    df['Price change'] = (df['Close'].pct_change() * 100).round(2)
    df['Volume change'] = (df['Volume'].pct_change() * 100).round(2)
    df['Date'] = df['Date'].apply(convert_date)
    df.rename(columns={
    'Close %': 'Price'
}, inplace=True)
    return df


dataframes = [
                cocoa,
                cofee,
                copper,
                corn,
                cotton,  
                gold, 
                palladium, 
                platinum, 
                silver,
                soybean,
                wheat]


dataframes_with_preprocess = [preprocess_df(df) for df in dataframes]



In [5]:
dataframes_2 = [aluminium,
                zinc,
                nickel,
                gasoline,
                brent,
                wti ]

def preprocess_df_2(df):
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    df['Change %'] = df['Change %'].astype(str).str.replace('%', '').replace('nan', pd.NA).astype('float')
    df['Vol.'] = df['Vol.'].astype(str).replace({'K': '*1e3','M': '*1e6', 'nan': '0'}, regex=True).map(pd.eval).astype('float')
    df['Volume change'] = (df['Vol.'].pct_change() * 100).round(2)
    df.rename(columns={
    'Change %': 'Price change',
    'Vol.': 'Volume'
}, inplace=True)


dataframes_with_preprocess_2 = [preprocess_df_2(df) for df in dataframes_2]

In [6]:
cocoa.tail()

,Date,Close,Volume,Price change,Volume change
4832,2024-03-25,9649.0,19698,7.94,20.09
4833,2024-03-26,9622.0,26263,-0.28,33.33
4834,2024-03-27,9843.0,23291,2.30,-11.32
4835,2024-03-28,9766.0,17953,-0.78,-22.92
4836,2024-04-01,10120.0,11594,3.62,-35.42


In [7]:
def fill_with_avg_of_neighbors(df):
    for column_name in df.columns:
        series = df[column_name]
        if pd.api.types.is_numeric_dtype(series):
           
            forward_filled = series.ffill()
            backward_filled = series.bfill()
            
            averaged_series = (forward_filled + backward_filled) / 2
            df.loc[df.index[:-1], column_name] = series.iloc[:-1].fillna(averaged_series[:-1])
            if pd.isna(series.iloc[-1]):
                df.loc[df.index[-1], column_name] = forward_filled.iloc[-1]

    return df

dataframes = [aluminium, 
brent ,
cocoa ,
cofee,
copper,
corn ,
cotton, 
gasoline, 
gold,
nickel, 
palladium,  
platinum, 
silver,
soybean,
wheat, 
wti,
zinc]

for i, df in enumerate(dataframes):
    dataframes[i] = fill_with_avg_of_neighbors(df)

In [8]:
nan_rows = aluminium[aluminium.isna().any(axis=1)]
print(nan_rows)


         Date     Price  Volume  Price change  Volume change
0  2024-04-04  2,449.00     0.0          0.82            NaN
1  2024-04-03  2,429.00     0.0          2.06            NaN


In [9]:
cocoa.tail()

,Date,Close,Volume,Price change,Volume change
4832,2024-03-25,9649.0,19698,7.94,20.09
4833,2024-03-26,9622.0,26263,-0.28,33.33
4834,2024-03-27,9843.0,23291,2.30,-11.32
4835,2024-03-28,9766.0,17953,-0.78,-22.92
4836,2024-04-01,10120.0,11594,3.62,-35.42


In [10]:
all_dfs = [aluminium, 
brent ,
cocoa ,
cofee,
copper,
corn ,
cotton, 
gasoline, 
gold,
nickel, 
palladium,  
platinum, 
silver,
soybean,
wheat, 
wti,
zinc]

In [11]:
def filter_by_date(df, date_column, start_date, end_date):
    df[date_column] = pd.to_datetime(df[date_column])
    filtered_df = df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]
    return filtered_df


date_column = 'Date'
start_date = '2007-06-01'
end_date = '2024-03-15'

aluminium = filter_by_date(aluminium, date_column, start_date, end_date)
brent = filter_by_date(brent, date_column, start_date, end_date)
cocoa = filter_by_date(cocoa, date_column, start_date, end_date)
cofee = filter_by_date(cofee, date_column, start_date, end_date)
copper = filter_by_date(copper, date_column, start_date, end_date)
corn = filter_by_date(corn, date_column, start_date, end_date)
cotton = filter_by_date(cotton, date_column, start_date, end_date)
gasoline = filter_by_date(gasoline, date_column, start_date, end_date)
gold = filter_by_date(gold, date_column, start_date, end_date)
nickel = filter_by_date(nickel, date_column, start_date, end_date)
palladium = filter_by_date(palladium, date_column, start_date, end_date)
platinum = filter_by_date(platinum, date_column, start_date, end_date)
silver = filter_by_date(silver, date_column, start_date, end_date)
soybean = filter_by_date(soybean, date_column, start_date, end_date)
wheat = filter_by_date(wheat, date_column, start_date, end_date)
wti = filter_by_date(wti, date_column, start_date, end_date)
zinc = filter_by_date(zinc, date_column, start_date, end_date)

In [12]:
aluminium = aluminium.iloc[::-1].reset_index(drop=True)
zinc = zinc.iloc[::-1].reset_index(drop=True)
nickel = nickel.iloc[::-1].reset_index(drop=True)
gasoline = gasoline.iloc[::-1].reset_index(drop=True)
brent = brent.iloc[::-1].reset_index(drop=True)
wti = wti.iloc[::-1].reset_index(drop=True)

In [13]:
return_df = brent[['Date']]
return_df = return_df.merge(brent[['Date', 'Price change']], on='Date', how='left', suffixes=('_brent'))
return_df = return_df.merge(cocoa[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_cocoa'))
return_df = return_df.merge(cofee[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_cofee'))
return_df = return_df.merge(copper[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_copper'))
return_df = return_df.merge(corn[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_corn'))
return_df = return_df.merge(cotton[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_cotton'))
return_df = return_df.merge(gasoline[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_gasoline'))
return_df = return_df.merge(gold[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_gold'))
return_df = return_df.merge(nickel[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_nickel'))
return_df = return_df.merge(palladium[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_palladium'))
return_df = return_df.merge(silver[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_silver'))
return_df = return_df.merge(soybean[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_soybean'))
return_df = return_df.merge(wheat[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_wheat'))
return_df = return_df.merge(wti[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_wti'))
return_df = return_df.merge(zinc[['Date', 'Price change']], on='Date', how='left', suffixes=('', '_zinc'))
return_df['brent_log'] = np.log(return_df['Price change']/100 + 1)

return_df.head()

/var/folders/gk/lwvxlf616ggf8j4h0y1hjl6c0000gn/T/ipykernel_78228/1671314584.py:2: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  return_df = return_df.merge(brent[['Date', 'Price change']], on='Date', how='left', suffixes=('_brent'))


,Date,Price change,Price change_cocoa,Price change_cofee,Price change_copper,Price change_corn,Price change_cotton,Price change_gasoline,Price change_gold,Price change_nickel,Price change_palladium,Price change_silver,Price change_soybean,Price change_wheat,Price change_wti,Price change_zinc,brent_log
0,2007-06-01,1.51,0.00,4.92,0.27,-0.90,-1.12,-0.29,1.54,-4.64,1.14,2.03,1.40,0.73,1.67,2.03,0.014987
1,2007-06-04,1.93,-1.43,-1.79,1.85,-0.78,-0.97,-0.29,-0.07,0.67,0.01,0.04,-0.21,-0.10,1.74,1.67,0.019116
2,2007-06-05,0.07,1.35,1.08,-0.55,-0.91,3.52,-1.37,-0.16,-2.27,-1.58,0.52,1.53,1.35,-0.91,-0.71,0.000700
3,2007-06-06,0.81,-0.32,-0.09,-1.49,-1.45,-0.39,-0.77,-0.04,-1.18,0.90,-0.68,-0.72,-1.42,0.53,-2.26,0.008067
4,2007-06-07,0.28,-1.60,0.43,-0.53,2.67,0.41,0.11,-1.37,-5.20,-0.60,-1.68,1.19,0.91,1.47,-0.79,0.002796


In [14]:
return_df.columns

Index(['Date', 'Price change', 'Price change_cocoa', 'Price change_cofee',
       'Price change_copper', 'Price change_corn', 'Price change_cotton',
       'Price change_gasoline', 'Price change_gold', 'Price change_nickel',
       'Price change_palladium', 'Price change_silver', 'Price change_soybean',
       'Price change_wheat', 'Price change_wti', 'Price change_zinc',
       'brent_log'],
      dtype='object')

In [15]:
return_df.drop('brent_log', axis = 1, inplace=True)
return_df['brent_log'] = np.log(return_df['Price change']/100 + 1) 
return_df['cocoa_log'] = np.log(return_df['Price change_cocoa']/100 + 1) 
return_df['cofee_log'] = np.log(return_df['Price change_cofee']/100 + 1)
return_df['copper_log'] = np.log(return_df['Price change_copper']/100 + 1)
return_df['corn_log'] = np.log(return_df['Price change_corn']/100 + 1) 
return_df['cotton_log'] = np.log(return_df['Price change_cotton']/100 + 1) 
return_df['gasoline_log'] = np.log(return_df['Price change_gasoline']/100 + 1) 
return_df['gold_log'] = np.log(return_df['Price change_gold']/100 + 1)
return_df['nickel_log'] = np.log(return_df['Price change_nickel']/100 + 1) 
return_df['palladium_log'] = np.log(return_df['Price change_palladium']/100 + 1) 
return_df['silver_log'] = np.log(return_df['Price change_silver']/100 + 1)
return_df['soybean_log'] = np.log(return_df['Price change_soybean']/100 + 1)
return_df['wheat_log'] = np.log(return_df['Price change_wheat']/100 + 1)
return_df['wti_log'] = np.log(return_df['Price change_wti']/100 + 1)
return_df['zinc_log'] = np.log(return_df['Price change_zinc']/100 + 1)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:

'''
return_df['aluminium_log'] = np.log(aluminium['Price change']/100 + 1)
return_df['brent_log'] = np.log(brent['Price change']/100 + 1) 
return_df['cocoa_log'] = np.log(cocoa['Price change']/100 + 1) 
return_df['cofee_log'] = np.log(cofee['Price change']/100 + 1)
return_df['copper_log'] = np.log(copper['Price change']/100 + 1)
return_df['corn_log'] = np.log(corn['Price change']/100 + 1) 
return_df['cotton_log'] = np.log(cotton['Price change']/100 + 1) 
return_df['gasoline_log'] = np.log(gasoline['Price change']/100 + 1) 
return_df['gold_log'] = np.log(gold['Price change']/100 + 1)
return_df['nickel_log'] = np.log(nickel['Price change']/100 + 1) 
return_df['palladium_log'] = np.log(palladium['Price change']/100 + 1) 
return_df['platinum_log'] = np.log(platinum['Price change']/100 + 1) 
return_df['silver_log'] = np.log(silver['Price change']/100 + 1)
return_df['soybean_log'] = np.log(soybean['Price change']/100 + 1)
return_df['wheat_log'] = np.log(wheat['Price change']/100 + 1)
return_df['wti_log'] = np.log(wti['Price change']/100 + 1)
return_df['zinc_log'] = np.log(zinc['Price change']/100 + 1)
'''

"\nreturn_df['aluminium_log'] = np.log(aluminium['Price change']/100 + 1)\nreturn_df['brent_log'] = np.log(brent['Price change']/100 + 1) \nreturn_df['cocoa_log'] = np.log(cocoa['Price change']/100 + 1) \nreturn_df['cofee_log'] = np.log(cofee['Price change']/100 + 1)\nreturn_df['copper_log'] = np.log(copper['Price change']/100 + 1)\nreturn_df['corn_log'] = np.log(corn['Price change']/100 + 1) \nreturn_df['cotton_log'] = np.log(cotton['Price change']/100 + 1) \nreturn_df['gasoline_log'] = np.log(gasoline['Price change']/100 + 1) \nreturn_df['gold_log'] = np.log(gold['Price change']/100 + 1)\nreturn_df['nickel_log'] = np.log(nickel['Price change']/100 + 1) \nreturn_df['palladium_log'] = np.log(palladium['Price change']/100 + 1) \nreturn_df['platinum_log'] = np.log(platinum['Price change']/100 + 1) \nreturn_df['silver_log'] = np.log(silver['Price change']/100 + 1)\nreturn_df['soybean_log'] = np.log(soybean['Price change']/100 + 1)\nreturn_df['wheat_log'] = np.log(wheat['Price change']/100

In [17]:
"""aluminium,
                zinc,
                nickel,
                gasoline,
                brent,
                wti


cocoa,
                cofee,
                copper,
                corn,
                cotton,  
                gold, 
                palladium, 
                platinum, 
                silver,
                soybean,
                wheat"""

'aluminium,\n                zinc,\n                nickel,\n                gasoline,\n                brent,\n                wti\n\n\ncocoa,\n                cofee,\n                copper,\n                corn,\n                cotton,  \n                gold, \n                palladium, \n                platinum, \n                silver,\n                soybean,\n                wheat'

In [18]:
return_df[['Date', 'Price change','Price change_cocoa']].tail()

,Date,Price change,Price change_cocoa
4332,2024-03-11,0.16,4.74
4333,2024-03-12,-0.35,9.81
4334,2024-03-13,2.58,-0.17
4335,2024-03-14,1.65,-7.83
4336,2024-03-15,-0.09,8.28


In [19]:
return_df.columns

Index(['Date', 'Price change', 'Price change_cocoa', 'Price change_cofee',
       'Price change_copper', 'Price change_corn', 'Price change_cotton',
       'Price change_gasoline', 'Price change_gold', 'Price change_nickel',
       'Price change_palladium', 'Price change_silver', 'Price change_soybean',
       'Price change_wheat', 'Price change_wti', 'Price change_zinc',
       'brent_log', 'cocoa_log', 'cofee_log', 'copper_log', 'corn_log',
       'cotton_log', 'gasoline_log', 'gold_log', 'nickel_log', 'palladium_log',
       'silver_log', 'soybean_log', 'wheat_log', 'wti_log', 'zinc_log'],
      dtype='object')